In [ ]:
# 🔑 Configure sua API Key da OpenRouter (opcional)
# Se não configurar, o sistema funcionará sem monitoramento LLM

OPENROUTER_API_KEY = ""  # @param {type:"string"}

# Configurar como variável de ambiente se fornecida
import os
if OPENROUTER_API_KEY.strip():
    os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY
    print("✅ API Key OpenRouter configurada - Monitor LLM ativado")
else:
    print("⚠️  API Key não configurada - Monitor LLM desabilitado")
    print("   O treinamento funcionará normalmente sem monitoramento inteligente")


In [ ]:
# Verificar GPU disponível
!nvidia-smi

# Conectar ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navegar para diretório de trabalho
%cd /content


In [ ]:
# Clone do repositório ValeTTS atualizado
!git clone https://github.com/wallacemt/ValeTTS.git
%cd ValeTTS

# Verificar estrutura do projeto
print("📁 Estrutura do projeto:")
!ls -la


In [ ]:
# Instalar dependências do projeto
!pip install -r requirements.txt

# Instalar o projeto em modo desenvolvimento
!pip install -e .

print("✅ Dependências instaladas com sucesso!")


In [ ]:
# Criar diretório de dados
!mkdir -p data/generated

# Extrair dataset do Google Drive
import os
dataset_path = "/content/drive/MyDrive/ValeTTS-Colab/Dataset-Unificado.tar.gz"

if os.path.exists(dataset_path):
    print("📦 Extraindo dataset...")
    !tar -xzf "/content/drive/MyDrive/ValeTTS-Colab/Dataset-Unificado.tar.gz" -C data/generated/

    # Verificar extração
    print("\n📁 Conteúdo do dataset:")
    !ls -la data/generated/Dataset-Unificado/

    # Verificar metadados
    print("\n📋 Arquivos de metadados:")
    !find data/generated/Dataset-Unificado/ -name "*.json" -o -name "*.csv" | head -5

    print("\n✅ Dataset restaurado com sucesso!")
else:
    print(f"❌ Dataset não encontrado em: {dataset_path}")
    print("   Verifique se o arquivo existe no Google Drive")


In [ ]:
# Testar se o script de treinamento funciona
print("🧪 Testando sistema de treinamento...")
!python scripts/train_vits2.py --help

print("\n✅ Sistema funcionando corretamente!")


In [ ]:
# Mostrar configuração que será usada
import yaml

with open('configs/training/vits2_dataset_unificado.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("📋 Configuração do Treinamento:")
print(f"   • Dataset: {config['data']['dataset_path']}")
print(f"   • Speakers: {config['model']['num_speakers']}")
print(f"   • Epochs: {config['training']['epochs']}")
print(f"   • Batch Size: {config['training']['batch_size']}")
print(f"   • Learning Rate: {config['training']['learning_rate']}")
print(f"   • Mixed Precision: {config['training']['mixed_precision']}")

if 'llm_monitor' in config:
    print(f"   • Monitor LLM: {config['llm_monitor'].get('enabled', False)}")

print("\n🎯 Configuração carregada com sucesso!")


In [ ]:
# 🎯 COMANDO EXATO DO AMBIENTE LOCAL
# python scripts/train_vits2.py --config configs/training/vits2_dataset_unificado.yaml

import os

# Determinar se deve desabilitar LLM baseado na API key
disable_llm = "" if os.environ.get('OPENROUTER_API_KEY') else "--disable-llm"

print("🚀 Iniciando treinamento VITS2...")
print(f"📁 Configuração: configs/training/vits2_dataset_unificado.yaml")
print(f"🔧 Monitor LLM: {'Ativado' if not disable_llm else 'Desabilitado'}")
print("\n" + "="*50)

# Executar treinamento
!python scripts/train_vits2.py --config configs/training/vits2_dataset_unificado.yaml {disable_llm}


In [ ]:
# Carregar TensorBoard para monitoramento
%load_ext tensorboard
%tensorboard --logdir logs/tensorboard/ --port 6006


In [ ]:
# Verificar progresso do treinamento
print("📊 Status do Treinamento:")
!ls -la logs/

print("\n📁 Checkpoints:")
!find logs/ -name "*.ckpt" | tail -5

print("\n📈 Logs mais recentes:")
!tail -10 logs/training.log 2>/dev/null || echo "Log não encontrado ainda"


In [ ]:
# Backup de modelos para Google Drive
import os
from shutil import copytree, ignore_patterns

backup_dir = "/content/drive/MyDrive/ValeTTS-Colab/training_backup"

if os.path.exists("logs/"):
    print("💾 Fazendo backup dos logs e checkpoints...")
    !mkdir -p "{backup_dir}"
    !cp -r logs/ "{backup_dir}/" 2>/dev/null || echo "Backup parcial realizado"
    print(f"✅ Backup salvo em: {backup_dir}")
else:
    print("ℹ️  Nenhum log encontrado para backup")
